In [1]:
import nltk
import os
import pandas as pd


In [2]:
df = pd.read_csv('IMDB_Dataset.csv')
df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [3]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]
df
# check if there two values or it contains any other too 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:

x=df["review"]
df['sentiment']=df["sentiment"].replace({
  'positive' or 'Positive':1,
  'negative' or 'negative':0
  
})
df=df.sample(5000)


C:\Users\msurahman\AppData\Local\Temp\ipykernel_19652\2428369636.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment']=df["sentiment"].replace({


In [5]:
x=df['review']
y=df['sentiment']

In [14]:
df["sentiment"].value_counts()

sentiment
0    2501
1    2499
Name: count, dtype: int64

In [6]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from revised_stopwords import get_revised_stopwords
stopwords_list = get_revised_stopwords()
nltk.download('punkt_tab')
print(stopwords_list)


{'those', 'has', 'when', 'yourselves', 'we', 'them', 'other', 'y', "you've", 'than', 'her', 'herself', "she'll", 'now', 'through', "i'll", "they'll", 'been', 'my', "she'd", 'she', 'his', 'an', 'some', "it'll", "we'll", 'whom', 'over', 'which', 'down', 'out', 'from', 'i', 'on', 'during', 'me', 'in', 've', 'what', "they'd", 'few', 'further', 'this', 'once', 'your', "he's", 'do', 'where', "they've", "he'd", "i'm", 'him', 'above', 'at', 're', 'yourself', 'with', 'be', 'that', 'until', 'was', 'did', "we've", "it'd", 'too', 'it', 'such', 'after', 'again', 'our', 'there', 'who', 'being', 'they', 'by', 'if', 'does', 'each', 'he', "it's", 'about', 'between', 'is', 'were', 'of', "should've", "i've", 'only', 'own', 'so', 'yours', 'up', "we'd", 'm', "we're", 'or', 'had', 'd', 'ours', 'themselves', 'll', 'a', 'both', "he'll", 'the', 'you', 'but', 'all', 'himself', 'itself', 'because', 'same', 'are', 'and', 'for', 'just', 'doing', 'below', 'these', 'o', 's', 'ourselves', 'their', 'theirs', 'will', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msurahman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\msurahman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
stop_words=set(get_revised_stopwords())


In [8]:
def preprocessed_text(text):
  text=re.sub(r'<[^>]+>',' ',text)
  stop_words=set(get_revised_stopwords())
  tokens=word_tokenize(text)
  filtered_tokens=[token for token in tokens if token.lower() not in stop_words]
  stemmed_tokens=[PorterStemmer().stem(token) for token in filtered_tokens]
  preprocessed_text= ' '.join(stemmed_tokens)
  return preprocessed_text.lower()


In [9]:
x = [preprocessed_text(text) for text in x]
preprocessed_df=pd.DataFrame({"review":x,"sentiment":y})
preprocessed_df.head(4)


,review,sentiment
16034,saw film ( 's english titl `` 's sing ? '' ) 1...,1
24819,ultra cheezi soundtrack . vinni tri realli har...,0
44882,"live mexico citi , suffer throug trailer everi...",0
49156,"watch movi scifi channel , conclud film made b...",0


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import mlflow
import logging
import os
import time


In [15]:
train_texts,test_texts,train_labels,test_labels=train_test_split(x,y,test_size=0.05,random_state=42)
num_words=1000
maxlen=500
tokenizer=Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x)


train_sequences=tokenizer.texts_to_sequences(train_texts)
test_sequences=tokenizer.texts_to_sequences(test_texts)
# we have to give a fixed length of words to an algo that is why we are applying this
train_data=pad_sequences(train_sequences,maxlen=maxlen)
test_data=pad_sequences(test_sequences,maxlen=maxlen)

test_data

array([[  0,   0,   0, ..., 429, 908, 186],
       [  0,   0,   0, ...,  68, 706,  82],
       [  0,   0,   0, ..., 146, 420,  52],
       ...,
       [  0,   0,   0, ...,   1,  71,   8],
       [  0,   0,   0, ..., 224, 971, 135],
       [  0,   0,   0, ..., 172, 376, 320]], shape=(250, 500), dtype=int32)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 16034 to 34895
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     5000 non-null   object
 1   sentiment  5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


In [13]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/MSURahmands365/end_to_end_mlops_pr.mlflow')
dagshub.init(repo_owner='MSURahmands365', repo_name='end_to_end_mlops_pr', mlflow=True)
mlflow.set_experiment("CNN & LSTM Base Line")

Initialized MLflow to track repo "MSURahmands365/end_to_end_mlops_pr"

Repository MSURahmands365/end_to_end_mlops_pr initialized!

2026/01/27 12:58:32 INFO mlflow.tracking.fluent: Experiment with name 'CNN & LSTM Base Line' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/df722788da374734bcc27f7e948e226a', creation_time=1769500712694, experiment_id='1', last_update_time=1769500712694, lifecycle_stage='active', name='CNN & LSTM Base Line', tags={}>

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout,Bidirectional
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
# 1.  Model Definition
logging.info("Starting MLflow run...")
with mlflow.start_run():
    start_time = time.time()
    try:
        model = Sequential([
            Embedding(input_dim=num_words, output_dim=128, input_length=maxlen),

            Conv1D(filters=128, kernel_size=5, activation='relu'),
            MaxPooling1D(pool_size=2),

            Bidirectional(LSTM(64)),

            Dense(1, activation='sigmoid')
        ])

        # 2. Compile
        model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

        # 3. Train
        history = model.fit(train_data, train_labels,
                            batch_size=128,
                            epochs=5,
                            validation_split=0.2)

        # 4. Evaluate
        loss, accuracy = model.evaluate(test_data, test_labels)
        print(f'Test accuracy: {accuracy:.4f}')
        y_predict=(model.predict(test_data)> 0.5).astype("int32")
        accuracy=accuracy_score(test_labels, y_predict)
        precision=precision_score(test_labels, y_predict)
        recall=recall_score(test_labels, y_predict)
        f1=f1_score(test_labels, y_predict)
        
        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_param("filter_size", 128)
        mlflow.log_param("lstm_units", 64)
        mlflow.log_param("epochs", 5)
        logging.info("Saving and logging the model...")
        mlflow.tensorflow.log_model(model, artifact_path="model")
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")
    except Exception as e:
        logging.error(f"An error Occured: {e}", exc_info=True)    

2026-01-27 15:28:13,211 - INFO - Starting MLflow run...


Epoch 1/5


c:\Users\msurahman\Documents\projects\End to End ML\end_to_end_mlops_pr\.venv\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


30/30 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.6189 - loss: 0.6397 - val_accuracy: 0.7821 - val_loss: 0.4720
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.8526 - loss: 0.3658 - val_accuracy: 0.8326 - val_loss: 0.3834
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9029 - loss: 0.2593 - val_accuracy: 0.8526 - val_loss: 0.3659
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9258 - loss: 0.2155 - val_accuracy: 0.8537 - val_loss: 0.3640
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.9397 - loss: 0.1817 - val_accuracy: 0.8495 - val_loss: 0.4007
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8640 - loss: 0.3587
Test accuracy: 0.8640
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step


2026-01-27 15:32:02,453 - INFO - Logging evaluation metrics...
2026-01-27 15:32:05,845 - INFO - Saving and logging the model...
2026/01/27 15:32:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/27 15:32:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2026/01/27 15:32:30 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026-01-27 15:32:39,590 - INFO - Model training and logging completed in 265.37 seconds.
2026-01-27 15:32:39,592 - INFO - Accuracy: 0.864
2026-01-27 15:32:39,594 - INFO - Precision: 0.8699186991869918
2026-01-27 15:32:39,597 - INFO - Recall: 0.856
2026-01-27 15:32:39,598 - INFO - F1 Score: 0.8629032258064516


🏃 View run valuable-owl-67 at: https://dagshub.com/MSURahmands365/end_to_end_mlops_pr.mlflow/#/experiments/1/runs/93636e7d42484765a3322d38a1481904
🧪 View experiment at: https://dagshub.com/MSURahmands365/end_to_end_mlops_pr.mlflow/#/experiments/1
